In [1]:
import json
import requests
import itertools
from collections import Counter, OrderedDict
import pandas as pd
%matplotlib inline
from tqdm import tqdm

In [2]:
import os
import pandas as pd
import glob
import json
import csv
import sys

import multiprocessing

import numpy as np
import re

from nltk.tokenize import wordpunct_tokenize

import requests
# from pprint import pprint
import pandas as pd

from collections import defaultdict, Counter
import time
from requests.compat import urljoin


In [3]:
from flair.models import TextClassifier
from flair.data import Sentence, Token
from flair.models import SequenceTagger

flair_models = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/flair_models/ner/manual_annotated_dataset/'
# load the model you trained
model = SequenceTagger.load(flair_models+'best-model.pt')



2020-01-30 11:09:22,194 loading file /nfs/gns/literature/Santosh_Tirunagari/GitHub/flair_models/ner/manual_annotated_dataset/best-model.pt


In [4]:
from nltk.tokenize import wordpunct_tokenize, WordPunctTokenizer

def custom_tokenizer(text):
    """
    Tokenizer based on word and punctuations only.
    """
    tokens: List[Token] = []

    tokenizer = WordPunctTokenizer()

    text = tokenizer.tokenize(text)

    index = 0
    for index, word in enumerate(text):
        tokens.append(
            Token(
                text=word, start_position=index, whitespace_after=False
            )
        )

    return tokens



In [5]:
from ast import literal_eval
from tqdm import tqdm

# result_path = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/PCSE/'
# result_file_name = 'OTAR-GP-DS-Data.csv'

In [6]:
def ML_annotate_GP_DS_in_sentences(complete_file_path):

    result_path = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/PCSE/'
    result_file_name = 'OTAR-GP-DS-Data.csv'

    GP_DS_data_annot = result_path+result_file_name
    
    annot_csv = pd.read_csv(complete_file_path,sep='\t')
    
    
    with open(GP_DS_data_annot, 'a', newline='\n') as f1:
        public_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

        for g, anno_df in tqdm(annot_csv.groupby(np.arange(len(annot_csv)) // 16)):

            otar_sentence = []
            pmc_id = []
            section = []
            GP_ner = []
            DS_ner = []
            sentences = []

    #         for index_, each_annotation in tqdm(enumerate(anno_df.itertuples()),total=len(anno_df)):
            for each_annotation in anno_df.itertuples():
                
                if len(each_annotation.sentence) < 600:
                    pmc_id.append(each_annotation.pmcid)
                    section.append(each_annotation.section)
                    otar_sentence.append(each_annotation.sentence)
                    GP_ner.append(each_annotation.GP)
                    DS_ner.append(each_annotation.DS)
                    sentences.append(Sentence(each_annotation.sentence, use_tokenizer=custom_tokenizer)) 

            predicted_sentences = model.predict(sentences, mini_batch_size=16)


            for i in range(0,len(otar_sentence)):
                entities = predicted_sentences[i].to_dict(tag_type='ner')['entities']
                DS_entities = []
                GP_entities = []
                
                if entities:
                    tagged_sents = predicted_sentences[i].to_dict(tag_type='ner')
                    for root_node in tagged_sents['entities']:
                        exact = root_node['text']
                        entity = root_node['type']

                        if entity == 'DS':    
                            DS_entities.append(exact)  
                        if entity == 'GP':    
                            GP_entities.append(exact)     

                predicted_ner = [pmc_id[i], section[i], otar_sentence[i], GP_ner[i], GP_entities, DS_ner[i], DS_entities]
                public_writer.writerow(predicted_ner)
            del anno_df
        del annot_csv

In [ ]:
file_path = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/Final_data/'

split_list_ = sorted(glob.glob(os.path.join(file_path, '*split*'))) 
split_list = split_list_[14:60]

for each_file in tqdm(split_list, total = len(split_list)):
    ML_annotate_GP_DS_in_sentences(each_file)

  5%|▍         | 152/3098 [01:38<31:53,  1.54it/s]


 10%|▉         | 303/3098 [03:16<29:32,  1.58it/s]


 15%|█▍        | 454/3098 [04:53<32:15,  1.37it/s]


 20%|█▉        | 605/3098 [06:31<28:28,  1.46it/s]


 24%|██▍       | 756/3098 [08:09<31:15,  1.25it/s]


...

 77%|███████▋  | 2420/3146 [27:59<08:45,  1.38it/s]


 82%|████████▏ | 2568/3146 [29:44<07:28,  1.29it/s]


 86%|████████▋ | 2716/3146 [31:41<05:31,  1.30it/s]


 91%|█████████ | 2864/3146 [33:31<03:03,  1.54it/s]


 93%|█████████▎| 2921/3146 [34:09<02:14,  1.68it/s]